In [ ]:
import streamlit as st
import pandas as pd
import mysql.connector
from mysql.connector import Error

# MySQL DB connection details
DB_HOST = "localhost"
DB_PORT = "3306"
DB_USER = "root"
DB_PASSWORD = "123456"
DB_NAME = "car"

# Connect to the MySQL database
def create_connection():
    try:
        connection = mysql.connector.connect(
            host=DB_HOST,
            port=DB_PORT,
            user=DB_USER,
            password=DB_PASSWORD,
            database=DB_NAME
        )
        return connection
    except Error as e:
        st.error(f"Error connecting to database: {e}")
        return None

# Insert data into the database
def insert_car_info(data):
    query = """
    INSERT INTO car_info (
        transmission, model, year_of_manufacture, city, ft, bt,
        ownership, kms_driven, engine_type
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    connection = create_connection()
    if connection:
        try:
            cursor = connection.cursor()
            cursor.execute(query, tuple(data.values()))
            connection.commit()
            cursor.close()
            connection.close()
            return True
        except Error as e:
            st.error(f"Failed to insert data: {e}")
            return False
    return False

# Streamlit app UI
st.title("Used Car Information Form")

with st.form("car_input_form"):
    transmission = st.selectbox("Transmission (0=Manual, 1=Automatic)", [0, 1])
    model = st.number_input("Model (numeric code)", min_value=0)
    year = st.number_input("Year of Manufacture", min_value=1900, max_value=2025, value=2015)
    city = st.selectbox("City (numeric code)", [0, 1, 2, 3, 4, 5])
    ft = st.number_input("ft (feature)", min_value=0)
    bt = st.number_input("bt (feature)", min_value=0)
    ownership = st.selectbox("Ownership (1=1st, 2=2nd, etc.)", [1, 2, 3, 4])
    kms_driven = st.number_input("Kms Driven", min_value=0)
    engine_type = st.number_input("Engine Type (numeric code)", min_value=0)

    submitted = st.form_submit_button("Submit")

if submitted:
    input_data = {
        "transmission": transmission,
        "model": model,
        "year_of_manufacture": year,
        "city": city,
        "ft": ft,
        "bt": bt,
        "ownership": ownership,
        "kms_driven": kms_driven,
        "engine_type": engine_type
    }

    if insert_car_info(input_data):
        st.success("Data successfully inserted into database!")
        st.write("You entered:")
        st.dataframe(pd.DataFrame([input_data]))
    else:
        st.error("Failed to insert data.")

